# Notebook 03: Functional Connectivity

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from nilearn.connectome import ConnectivityMeasure
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported")

In [ ]:
np.random.seed(42)
print("✓ Random seed set to 42")

In [ ]:
notebook_dir = Path.cwd()
project_dir = notebook_dir.parent
data_dir = project_dir / 'data'
figures_dir = project_dir / 'figures'

config_path = data_dir / 'config.json'
with open(config_path, 'r') as f:
    config = json.load(f)

print(f"✓ Configuration loaded")

In [ ]:
timeseries_path = data_dir / 'all_time_series.npy'
all_time_series = np.load(timeseries_path, allow_pickle=True)
print(f"✓ Time series loaded")
print(f"  Number of subjects: {len(all_time_series)}")

In [ ]:
print("Calculating correlation matrices...")
connectivity_measure = ConnectivityMeasure(kind='correlation')

subject_matrices = []
for i, time_series in enumerate(all_time_series):
    corr_matrix = connectivity_measure.fit_transform([time_series])[0]
    subject_matrices.append(corr_matrix)

print(f"✓ Correlation matrices computed for {len(subject_matrices)} subjects")

In [ ]:
mean_correlation = np.mean(subject_matrices, axis=0)

print(f"✓ Group-averaged correlation matrix computed")
print(f"  Matrix shape: {mean_correlation.shape}")
print(f"  Value range: [{mean_correlation.min():.4f}, {mean_correlation.max():.4f}]")

In [ ]:
functional_matrix_path = data_dir / 'functional_matrix.npy'
np.save(functional_matrix_path, mean_correlation)

print(f"✓ Functional connectivity matrix saved")

In [ ]:
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['savefig.dpi'] = 300

fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(
    mean_correlation,
    cmap='RdBu_r',
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    cbar_kws={'label': 'Correlation Coefficient'},
    ax=ax,
    xticklabels=False,
    yticklabels=False
)

ax.set_title('Group-Averaged Functional Connectivity Matrix', fontsize=10, fontweight='bold')
plt.tight_layout()

pdf_path = figures_dir / 'functional_connectivity_matrix.pdf'
plt.savefig(pdf_path, dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Functional connectivity heatmap saved")

In [ ]:
print("\nTop 10 Strongest Connections:")
row_indices, col_indices = np.triu_indices_from(mean_correlation, k=1)
correlations = mean_correlation[row_indices, col_indices]

connections_df = pd.DataFrame({
    'Region1': row_indices,
    'Region2': col_indices,
    'Correlation': correlations
})

top_connections = connections_df.nlargest(10, 'Correlation')

for idx, row in top_connections.iterrows():
    print(f"  Region {int(row['Region1']):3d} <-> Region {int(row['Region2']):3d}: {row['Correlation']:.4f}")

print("\n✓ Functional Connectivity Complete!")